In [1]:
import warnings
warnings.filterwarnings('ignore')
import sqlite3
import requests
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import os
import selenium
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import random
import undetected_chromedriver as uc

In [2]:
# Susirenkame varle.lt dronų nuorodas.

opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)

nuorodos=[]

for i in range(1, 4):
    url = f'https://www.varle.lt/dronai/?p={i}' 
    driver.get(url)
    time.sleep(2)
    source = driver.page_source
    bs = BeautifulSoup(source, 'html.parser')
    varle = bs.find_all('div', {'class':'GRID_ITEM'})

    for dronai in varle:
        try:
            nuorodos.append(dronai.find('div', {'class':'product-title'}).find('a')['href'])
        except:
            pass
        
driver.close()

In [5]:
def getColNamesList(table, dbname):
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''select * from {table} limit 1'''.format(table=table)
    s = c.execute(sql)
    r = s.description
    return r

def alterTableAddColumn(cols_i_want_to_have, cols_i_have, table, base):
    db = sqlite3.connect(base)
    c = db.cursor()
    sql = '''alter table {tname}
    add column `{n}`
    '''
    for cl in cols_i_want_to_have:
        if cl not in cols_i_have:
            c.execute(sql.format(tname=table, n=cl))
            time.sleep(0.25)
            db.commit()
            # time.sleep(3)
    c.close()
    db.close()
    return True

def writeMany(data, table, dbname):
    connection = sqlite3.connect(dbname)
    cursor = connection.cursor()

    # Insert dictionary data into the table
    # columns = ', '.join(data.keys())  # Get column names as a string
    placeholders = ', '.join(['?'] * len(data))  # Generate placeholders for values
    values = tuple(data.values())  # Get values as a tuple

    # Construct the SQL INSERT statement
    sql = f"INSERT INTO {table} VALUES ({placeholders})"
    cursor.execute(sql, values)

    # Commit the changes and close the connection
    connection.commit()
    cursor.close()
    connection.close()
    return True


SDB = sqlite3.connect('varledronai.db') # jei neegsiztuoja - bus sukurfta nauja sqlite DB
Cs = SDB.cursor()

sql = '''create table if not exists dronai
(
gamintojas text,
nuoroda text
)
 '''
Cs.execute(sql)
Cs.close()
SDB.close()



opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)

for nuor in range(0, len(nuorodos)):
    try:
        url = f'https://www.varle.lt{nuorodos[nuor]}'
        driver.get(url)
        time.sleep(2)
        source = driver.page_source
        bs = BeautifulSoup(source, 'html.parser')
        specifikacija = bs.find('div', {'class' : 'accordion'}).find('div', {'class': 'PRODUCT_SPEC'})
        duom=specifikacija.find_all('li', {'class': 'spec-line'})

        stulpeliai, *_ =zip(*getColNamesList('dronai', 'varledronai.db'))
        duomenys = {key: None for key in stulpeliai}
        duomenys['nuoroda']= nuorodos[nuor]
    
        kainaa = bs.find('div', {'class' : 'container'})
        kainaa= kainaa.find('span', {'class': 'price-value'})['content'].strip()
        duomenys['kaina'] = kainaa
    
        if len(duom) > 0:
            for i in duom:
                try:
                    stulpeliai, *_ =zip(*getColNamesList('dronai', 'varledronai.db'))
                    # print(i.find('span', {'class':'left-column'}).text.lower(), ' : ', i.find('span', {'class':'right-column'}).text.lower())
                    kas = i.find('span', {'class':'left-column'}).text.lower().strip()
                    kaip = i.find('span', {'class':'right-column'}).text.lower().strip()
                    duomenys[kas]= kaip
                except:
                    pass
        else:
            specifikacija1 = bs.find('div', {'class' : 'accordion'}).find('div', {'class': 'PRODUCT_SPEC'})
            duom1=specifikacija1.find_all('tr')

            for j in duom1:
                if len(j) == 2:
                    kas = j.find('td', {'class': 'key'}).text.lower().strip()
                    kaip = j.find('td', {'class': 'value'}).text.lower().strip()
                    duomenys[kas]= kaip
    
        try:
            specifikacija2 = bs.find('div', {'class' : 'accordion'}).find('div', {'class': 'PRODUCT_SPEC_FILTERS'})
            specifikacija2= specifikacija2.find_all('li', {'class': 'spec-line'})
            for i in specifikacija2:
                kas = i.find('span', {'class': 'left-column'}).text.lower().strip()
                kaip = i.find('span', {'class': 'right-column'}).text.lower().strip()
                duomenys[kas]=kaip
        except:
            pass
        
    except:
        stulpeliai, *_ =zip(*getColNamesList('dronai', 'varledronai.db'))
        alterTableAddColumn(duomenys.keys(), stulpeliai, 'dronai', 'varledronai.db')
        pass
    else:
        stulpeliai, *_ =zip(*getColNamesList('dronai', 'varledronai.db'))
        alterTableAddColumn(duomenys.keys(), stulpeliai, 'dronai', 'varledronai.db')
        writeMany(duomenys, 'dronai', 'varledronai.db')
    

driver.close()

In [2]:
# Susirenkame varle.lt dviračių nuorodas.

opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)

nuorodos=[]

for i in range(1, 55):
    url = f'https://www.varle.lt/dviraciai/?p={i}' 
    driver.get(url)
    time.sleep(2)
    source = driver.page_source
    bs = BeautifulSoup(source, 'html.parser')
    varle = bs.find_all('div', {'class':'GRID_ITEM'})

    for dronai in varle:
        try:
            nuorodos.append(dronai.find('div', {'class':'product-title'}).find('a')['href'])
        except:
            pass
        
driver.close()

In [4]:
def getColNamesList(table, dbname):
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''select * from {table} limit 1'''.format(table=table)
    s = c.execute(sql)
    r = s.description
    return r

def alterTableAddColumn(cols_i_want_to_have, cols_i_have, table, base):
    db = sqlite3.connect(base)
    c = db.cursor()
    sql = '''alter table {tname}
    add column `{n}`
    '''
    for cl in cols_i_want_to_have:
        if cl not in cols_i_have:
            c.execute(sql.format(tname=table, n=cl))
            time.sleep(0.25)
            db.commit()
            # time.sleep(3)
    c.close()
    db.close()
    return True

def writeMany(data, table, dbname):
    connection = sqlite3.connect(dbname)
    cursor = connection.cursor()

    # Insert dictionary data into the table
    # columns = ', '.join(data.keys())  # Get column names as a string
    placeholders = ', '.join(['?'] * len(data))  # Generate placeholders for values
    values = tuple(data.values())  # Get values as a tuple

    # Construct the SQL INSERT statement
    sql = f"INSERT INTO {table} VALUES ({placeholders})"
    cursor.execute(sql, values)

    # Commit the changes and close the connection
    connection.commit()
    cursor.close()
    connection.close()
    return True


SDB = sqlite3.connect('varledviratis.db') # jei neegsiztuoja - bus sukurfta nauja sqlite DB
Cs = SDB.cursor()

sql = '''create table if not exists dviratis
(
gamintojas text,
nuoroda text
)
 '''
Cs.execute(sql)
Cs.close()
SDB.close()



opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)

for nuor in range(0, len(nuorodos)):
    try:
        url = f'https://www.varle.lt{nuorodos[nuor]}'
        driver.get(url)
        time.sleep(2)
        source = driver.page_source
        bs = BeautifulSoup(source, 'html.parser')
        specifikacija = bs.find('div', {'class' : 'accordion'}).find('div', {'class': 'PRODUCT_SPEC'})
        duom=specifikacija.find_all('li', {'class': 'spec-line'})

        stulpeliai, *_ =zip(*getColNamesList('dviratis', 'varledviratis.db'))
        duomenys = {key: None for key in stulpeliai}
        duomenys['nuoroda']= nuorodos[nuor]
    
        kainaa = bs.find('div', {'class' : 'container'})
        kainaa= kainaa.find('span', {'class': 'price-value'})['content'].strip()
        duomenys['kaina'] = kainaa
    
        if len(duom) > 0:
            for i in duom:
                try:
                    stulpeliai, *_ =zip(*getColNamesList('dviratis', 'varledviratis.db'))
                    # print(i.find('span', {'class':'left-column'}).text.lower(), ' : ', i.find('span', {'class':'right-column'}).text.lower())
                    kas = i.find('span', {'class':'left-column'}).text.lower().strip()
                    kaip = i.find('span', {'class':'right-column'}).text.lower().strip()
                    duomenys[kas]= kaip
                except:
                    pass
        else:
            specifikacija1 = bs.find('div', {'class' : 'accordion'}).find('div', {'class': 'PRODUCT_SPEC'})
            duom1=specifikacija1.find_all('tr')

            for j in duom1:
                if len(j) == 2:
                    kas = j.find('td', {'class': 'key'}).text.lower().strip()
                    kaip = j.find('td', {'class': 'value'}).text.lower().strip()
                    duomenys[kas]= kaip
    
        try:
            specifikacija2 = bs.find('div', {'class' : 'accordion'}).find('div', {'class': 'PRODUCT_SPEC_FILTERS'})
            specifikacija2= specifikacija2.find_all('li', {'class': 'spec-line'})
            for i in specifikacija2:
                kas = i.find('span', {'class': 'left-column'}).text.lower().strip()
                kaip = i.find('span', {'class': 'right-column'}).text.lower().strip()
                duomenys[kas]=kaip
        except:
            pass
        
    except:
        stulpeliai, *_ =zip(*getColNamesList('dviratis', 'varledviratis.db'))
        alterTableAddColumn(duomenys.keys(), stulpeliai, 'dviratis', 'varledviratis.db')
        pass
    else:
        stulpeliai, *_ =zip(*getColNamesList('dviratis', 'varledviratis.db'))
        alterTableAddColumn(duomenys.keys(), stulpeliai, 'dviratis', 'varledviratis.db')
        writeMany(duomenys, 'dviratis', 'varledviratis.db')
    

driver.close()

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=131.0.6778.86)
Stacktrace:
	GetHandleVerifier [0x00DD33E3+25059]
	(No symbol) [0x00D5CDE4]
	(No symbol) [0x00C3BEC3]
	(No symbol) [0x00C2B21E]
	(No symbol) [0x00C2BF8D]
	(No symbol) [0x00C3C3F2]
	(No symbol) [0x00C1D555]
	(No symbol) [0x00CADAEE]
	(No symbol) [0x00CA1EC3]
	(No symbol) [0x00C73FAC]
	(No symbol) [0x00C74F3D]
	GetHandleVerifier [0x010C5543+3113795]
	GetHandleVerifier [0x010DA20A+3198986]
	GetHandleVerifier [0x010D29E2+3168226]
	GetHandleVerifier [0x00E73250+680016]
	(No symbol) [0x00D6572D]
	(No symbol) [0x00D629D8]
	(No symbol) [0x00D62B75]
	(No symbol) [0x00D557D0]
	BaseThreadInitThunk [0x75C97BA9+25]
	RtlInitializeExceptionChain [0x771CC0CB+107]
	RtlClearBits [0x771CC04F+191]


In [5]:
len(nuorodos)

2023